In [1]:
require 'torch'
require 'gnuplot'
require 'rnn'
require 'image'
Plot = require 'itorch.Plot'

In [2]:
require 'torch'
require 'nn'
require 'rnn'
require 'csvigo'

aurora_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_first_sentence_numtext.csv'
nugget_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/aurora_nuggets_numtext.csv'

m = csvigo.load({path = aurora_fn, mode = "large"})
q = csvigo.load({path = nugget_fn, mode = "large"})

N = 1000 --- Breaks at 35
K = 100
nepochs = 100
embed_dim = 6
cuda = true

torch.manualSeed(420)

dofile("utils.lua")

out = grabNsamples(m, N, K)             --- Extracting N samples
nggs = grabNsamples(q, #q-1, nil)       --- Extracting all samples

mxl = 0
for k,v in pairs(out) do
    mxl = math.max(mxl, #v)
end

vocab_size = 0                  --- getting the length of the dictionary
for k,v in pairs(out) do
    vocab_size = math.max(vocab_size, math.max(table.unpack(v)))
    if (k % N)==0 then
        print(k,'elements read out of ', #m)
        break
    end
end

function build_network(vocab_size, embed_dim, outputSize, cuda)
    batchLSTM = nn.Sequential()
    :add(nn.LookupTableMaskZero(vocab_size, embed_dim)) -- will return a sequence-length x batch-size x embedDim tensor
    :add(nn.SplitTable(1, embed_dim)) -- splits into a sequence-length table with batch-size x embedDim entries
    :add(nn.Sequencer(nn.LSTM(embed_dim, embed_dim)))
    :add(nn.SelectTable(-1)) -- selects last state of the LSTM
    :add(nn.Linear(embed_dim, outputSize)) -- map last state to a score for classification
    :add(nn.ReLU())

   return batchLSTM

end

xs = padZeros(out, mxl)             --- Padding the data by the maximum length
input = torch.LongTensor(xs)        --- This is the correct format to input it
labels = torch.rand(#out)

-- For batch inputs, it's a little easier to start with sequence-length x batch-size tensor, so we transpose songData
myDataT = input:t()
batchLSTM = build_network(vocab_size, embed_dim, 1, true)
crit = nn.MSECriterion()

err = {}
loss = 0
for epoch=1, nepochs, 1 do
    myPreds = batchLSTM:forward(myDataT)
    loss = loss + crit:forward(myPreds, labels)
    grads = crit:backward(myPreds, labels)
    batchLSTM:backward(myDataT, grads)
    err[epoch] = loss
    
    --We update params at the end of each batch
    batchLSTM:updateParameters(0.1)
    batchLSTM:zeroGradParameters()
    
    preds = {}
    for i=1, myPreds:size()[1] do
        preds[i] = (myPreds[i][1] > 0) and 1 or 0 --- lua is stupid
    end

    ys = unpackZeros(preds)
    predsummary = buildPredSummary(ys, xs)

    rscore = rougeRecall(predsummary, nggs)
    pscore = rougePrecision(predsummary, nggs)
    fscore = rougeF1(predsummary, nggs)
    print(string.format("Epoch %i, Rouge \t {Recall = %.6f, Precision = %6.f, F1 = %.6f}", epoch, rscore, pscore, fscore))
end

<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_first_sentence_numtext.csv	


<csv>	parsing done	
<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/aurora_nuggets_numtext.csv	
<csv>	parsing done	
...Utils file loaded	


1000	elements read out of 	12510	


139	elements read out of 	140	
1000	elements read out of 	12510	


Epoch 1, Rouge 	 {Recall = 0.955330, Precision =      0, F1 = 0.167961}	


...

Epoch 96, Rouge 	 {Recall = 0.960406, Precision =      0, F1 = 0.160149}	


Epoch 97, Rouge 	 {Recall = 0.960406, Precision =      0, F1 = 0.160149}	


Epoch 98, Rouge 	 {Recall = 0.960406, Precision =      0, F1 = 0.160149}	


Epoch 99, Rouge 	 {Recall = 0.960406, Precision =      0, F1 = 0.160149}	


Epoch 100, Rouge 	 {Recall = 0.960406, Precision =      0, F1 = 0.160149}	


In [4]:
function geti_n(x, n)
    local out = {}
    for k,v in pairs(x) do
        if k <= n then
        out[k] = v
        else
            break
        end
    end
    return out
end

In [8]:
rscores = {}
pscores = {}
fscores = {}
out = {}
for i=1, 4 do
    rscores[i] = rougeRecall(geti_n(predsummary, i), nggs)
    pscores[i] = rougePrecision(geti_n(predsummary, i), nggs)
    fscores[i] = rougeF1(geti_n(predsummary, i), nggs)
    print(string.format("{Recall = %.6f, Precision = %6.f, F1 = %.6f}", rscores[i], pscores[i], fscores[i]))
end

{Recall = 0.005076, Precision =      0, F1 = 0.010030}	
{Recall = 0.027411, Precision =      1, F1 = 0.052224}	


{Recall = 0.031472, Precision =      1, F1 = 0.059673}	
{Recall = 0.035533, Precision =      1, F1 = 0.066667}	


In [ ]:
rscore = rougeRecall(predsummary, nggs)
pscore = rougePrecision(predsummary, nggs)
fscore = rougeF1(predsummary, nggs)

In [ ]:
-- scatter plots
plot = Plot():circle(torch.range(1, #err), 
                err, 'red', 'RMSE'):draw()
plot:line(torch.range(1, #err), 
                err, 'red', ''):redraw()
plot:title('Plot of Loss Function'):redraw()
plot:xaxis('Iterations'):yaxis('Mean Squared Error'):redraw()
plot:legend(true)
plot:redraw()